In [1]:
from pathlib import Path
from dicomexport.dcm_io import read_dicom_file
import zipfile
from tqdm.notebook import tqdm
import shutil
import warnings

warnings.filterwarnings("ignore")


In [2]:
root = Path("samples/")  # Folder that contains all the zip files
output = Path("output")  # Folder where the extracted files will be saved

In [3]:
temp = Path("temp")
temp.mkdir(exist_ok=True)
output.mkdir(exist_ok=True)
all_zips = root.glob("*.zip")

print("Extracting zips")
for dcm_zip in tqdm(all_zips):
    with zipfile.ZipFile(dcm_zip, "r") as zip_ref:
        zip_ref.extractall(temp / "unzip" / dcm_zip.stem)

print("Reading dicom files")

for patient_folder in tqdm((temp / "unzip").glob("*")):
    folder = patient_folder / "DataFiles"
    patient_id = None
    laterality = None
    date = None
    for f in folder.iterdir():
        if f.is_dir():
            for file in f.iterdir():
                if file.is_file():
                    try:
                        laterality, date, patient_id = read_dicom_file(
                            file,
                            patientId=patient_id,
                            laterality=laterality,
                            date=date,
                            output_folder=temp / "extracted",
                        )
                    except Exception as e:
                        pass

print("Finding best matches")
matches = []
for d in tqdm((temp / "extracted").rglob("*.png")):
    parent_folder = d.parent
    if parent_folder in matches:
        continue
    size_folder = len(list(parent_folder.glob("*.png")))
    if size_folder == 128:
        metadata = str(d).split("/")
        patientId = metadata[2]
        date = metadata[3]
        laterality = metadata[4]

        out = output / patientId / date / laterality
        out.mkdir(parents=True, exist_ok=True)
        for img in parent_folder.glob("*.png"):
            shutil.copy(img, out / img.name)

        matches.append(parent_folder)


print("Cleaning up")
shutil.rmtree(temp)

Extracting zips


0it [00:00, ?it/s]

Reading dicom files


0it [00:00, ?it/s]

Finding best matches


0it [00:00, ?it/s]

Cleaning up
